# XML, HTML, and Web Scraping

JSON and XML are two different ways to represent hierarchical data. Which one is better? There are lots of articles online which discuss similarities and differences between JSON and XML and their advantages and disadvantages. Both formats are still in current usage, so it is good to be familiar with both. However, JSON is more common, so we'll focus on working with JSON representations of hierarchical data.

The reading covered an example of using Beautiful Soup to parse XML. Rather than doing another example XML now, we'll skip straight to scraping HTML from a webpage. Both HTML and XML can be parsed in a similar way with Beautiful Soup.

In [1]:
pip install beautifulsoup4

In [2]:
import pandas as pd

## Scraping an HTML table with Beautiful Soup

Open the URL https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population and scroll down until you see a table of the cities in the U.S. with population over 100,000 (as of Jul 1, 2022). We'll use Beautiful Soup to scrape information from this table.

Read in the HTML from the ULR using the `requests` library.

In [3]:
# YOUR CODE HERE
import requests
url = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64;x64)"}
response = requests.get(url, headers=HEADERS)

Use Beautiful Soup to parse this string into a tree called `soup`

In [4]:
# YOUR CODE HERE
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

To find an HTML tag corresponding to a specific element on a webpage, right-click on it and choose "Inspect element". Go to the cities table Wikipedia page and do this now.

You should find that the cities table on the Wikipedia page corresponds to the element

```
<table class="wikitable sortable jquery-tablesorter" style="text-align:center">

class="sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center jquery-tablesorter" style="text-align:right"
```

There are many `<table>` tags on the page.

In [5]:
len(soup.find_all("table"))

10

We can use attributes like `class=` and `style=` to narrow down the list.

In [6]:
len(soup.find_all("table",
                attrs={
                      "class": "sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center",
                      "style": "text-align:right"}
                  ))

1

At this point, you can manually inspect the tables on the webpage to find that the one we want is the first one (see `[0]` below). We'll store this as `table`.

In [8]:
right_tables = soup.find_all("table",
                  attrs={
                      "class": "sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center",
                      }
                  )

**Now you will write code to scrape the information in `table` to create a Pandas data frame with one row for each city and columns for: city, state, population (2022 estimate), and 2020 land area (sq mi).** Refer to the Notes/suggestions below as you write your code. A few Hints are provided further down, but try coding first before looking at the hints.

Notes/suggestions:

- Use as a guide the code from the reading that produced the data frame of Statistics faculty
- Inspect the page source as you write your code
- You will need to write a loop to get the information for all cities, but you might want to try just scraping the info for New York first
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `.get_text(strip = True)` instead of `.text`
- Don't forget to convert to a Pandas Data Frame; it should have 333 rows and 4 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it --- e.g., what is the population density for all cities in CA? --- then you would need to clean the data first (to clean strings and convert to quantitative). (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)

In [ ]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED

In [23]:
# initialize an empty list
rows = []
tbody = right_tables[0].find('tbody')
# iterate over all rows in the wiki table
for row in tbody.find_all("tr"):
  td = row.find_all("td")

  if len(td) == 0:
    continue

  # get all the cells (<td>) in the row
  # the information we need is the text between tags

  # find the name of the city in cell[0]
  city = td[0].get_text(strip = True)


  # find the state in cell[1]
  state = td[1].get_text(strip = True)


  # find the population in cell[2]
  population = td[2].get_text(strip = True)


  # find the land area in cell[5]
  land_area = td[5].get_text(strip=True)

  # append this data
  rows.append({
      "city": city,
      "state": state,
      "population": population,
      "land_area": land_area
  })

In [24]:
pd.DataFrame(rows)

,city,state,population,land_area
0,New York[c],NY,"8,478,072",300.5
1,Los Angeles,CA,"3,878,704",469.5
2,Chicago,IL,"2,721,308",227.7
3,Houston,TX,"2,390,125",640.4
4,Phoenix,AZ,"1,673,164",518.0
...,...,...,...,...
341,Deltona,FL,"100,513",37.3
342,Federal Way,WA,"100,252",22.3
343,San Angelo,TX,"100,159",59.7
344,Tracy,CA,"100,136",25.9


Hints:

- Each city is a row in the table; find all the `<tr>` tags to find all the cities
- Look for the `<td>` tag to see table entries within a row
- The rank column is represented by `<th>` tags, rather than `<td>` tags. So within a row, the first (that is, `[0]`) `<td>` tag corresponds to the city name.

## Aside: Scraping an HTML table with Pandas



The Pandas command `read_html` can be used to scrape information from an HTML table on a webpage.

We can call `read_html` on the URL.

In [ ]:
pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")

However, this scrapes all the tables on the webpage, not just the one we want. As with Beautiful Soup, we can narrow the search by specifying the table attributes.

In [ ]:
pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", attrs = {'class': 'wikitable sortable', "style": "text-align:center"})

This still returns 3 tables. As we remarked above, the table that we want is the first one (see `[0]` below).

In [ ]:
df_cities2 = pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", attrs = {'class': 'wikitable sortable', "style": "text-align:center"})[0]
df_cities2

Wait, that seemed much easier than using Beautiful Soup, and it returned a data frame, and we even got for free some formatting like removing the commas from the population! Why didn't we just use `read_html` in the first place? It's true the `read_html` works well when scraping information from an HTML *table*. Unfortunately, you often want to scrape information from a webpage that isn't conveniently stored in an HTML table, in which case `read_html` won't work. (It only searches for `<table>`, `<th>`, `<tr>`, and `<td>` tags, but there are many other HTML tags.) Though Beautiful Soup is not as simple as `read_html`, it is more flexible and thus more widely applicable.

## Scraping information that is NOT in a `<table>` with Beautiful Soup

The Cal Poly course catalog http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory contains a list of courses offered by the Statistics department. **You will scrape this website to obtain a Pandas data frame with one row for each DATA or STAT course and two columns: course name and number (e.g, DATA 301. Introduction to Data Science) and term typically offered (e.g., Term Typically Offered: F, W, SP).**

Note: Pandas `read_html` is not help here since the courses are not stored in a `<table>.`

In [28]:
pd.read_html("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")

[                                        Program name   Program type
 0                              Actuarial Preparation          Minor
 1  Cross Disciplinary Studies Minor in Bioinforma...          Minor
 2   Cross Disciplinary Studies Minor in Data Science          Minor
 3                                         Statistics  BS, MS, Minor]


Notes/suggestions:


- Inspect the page source as you write your code
- The courses are not stored in a `<table>`. How are they stored?
- You will need to write a loop to get the information for all courses, but you might want to try just scraping the info for DATA 100 first
- What kind of tag is the course name stored in? What is the `class` of the tag?
- What kind of tag is the quarter(s) the course is offered stored in? What is the `class` of the tag? Is this the only tag of this type with the class? How will you get the one you want?
- You don't have to remove the number of units (e.g., 4 units) from the course name and number, but you can try it if you want
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `get_text(strip = True)` instead of `text`
- Don't forget to convert to a Pandas Data Frame; it should have 74 rows and 2 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it then you might need to clean the data first. (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)



In [ ]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED

In [25]:
import requests
url = "https://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64;x64)"}
response = requests.get(url, headers=HEADERS)

In [26]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

In [29]:
course_columns = soup.find_all("div", {"class": "courseblock"})
course_columns

[<div class="courseblock">
 <p class="courseblocktitle"><strong>DATA 100. Data Science for All I.
 <span class="courseblockhours">4 units
 </span></strong></p><div class="noindent courseextendedwrap">
 <p class="noindent">Term Typically Offered: F, W, SP</p><p class="noindent">2020-21 or later catalog: GE Area B4</p><p class="noindent">2019-20 or earlier catalog: GE Area B4</p><p>Prerequisite: <a class="bubblelink code" href="/search/?P=MATH%20115" onclick="return showCourse(this, 'MATH 115');" title="MATH 115">MATH 115</a>, <a class="bubblelink code" href="/search/?P=MATH%20116" onclick="return showCourse(this, 'MATH 116');" title="MATH 116">MATH 116</a>, <a class="bubblelink code" href="/search/?P=MATH%20118" onclick="return showCourse(this, 'MATH 118');" title="MATH 118">MATH 118</a>, or Appropriate Math Placement Level.</p></div>
 <div class="courseblockdesc">
 <p>Basic approaches for answering questions using data.  Emphasis on working with tabular data in spreadsheet software to 

In [31]:
# initialize an empty list
rows = []

#iterate over all rows in the course table
for i in course_columns:
  title = i.find("p", class_="courseblocktitle")
  if not title:
    continue
  course_title = title.get_text(strip=True)

  school_term = ""

  for p in i.find_all("p"):
    if "Term Typically Offered:" in p.get_text(strip=True):
      school_term = p.get_text(strip=True).replace("Term Typically Offered:", "").strip()

  rows.append({
      "course_title": course_title,
      "school_term": school_term
  })


In [32]:
pd.DataFrame(rows)

,course_title,school_term
0,DATA 100. Data Science for All I.4 units,"F, W, SP"
1,DATA 301. Introduction to Data Science.4 units,"F, W, SP"
2,DATA 401. Data Science Process and Ethics.3 units,F
3,DATA 402. Mathematical Foundations of Data Sci...,F
4,DATA 403. Data Science Projects Laboratory.1 unit,F
...,...,...
69,STAT 551. Statistical Learning with R.4 units,F
70,STAT 566. Graduate Consulting Practicum.2 units,SP
71,STAT 570. Selected Advanced Topics.1-4 units,TBD
72,STAT 590. Graduate Seminar in Statistics.1 unit,"F, W, SP"


Hints:

- Each course is represented by a `<div>` with `class=courseblock`, so you can find all the courses with `soup.find_all("div", {"class": "courseblock"})`
- The course name is in a `<p>` tag with `class=courseblocktitle`, inside a `<strong>` tag. (Though I don't think we need to find the strong tag here.)
- The term typically offered is in `<p>` tag with `class=noindent`. However, there are several tags with this class; term typically offered is the first one.
- If you want to use Beautiful Soup to remove the course units (e.g., 4 units), find the `<span>` tag within the course name tag and `.extract()` this span tag